In [3]:
import numpy as np
import pandas as pd
import requests
import json

import sys
sys.path.append('../src/')
import wiki_api
import data_layer
import graph_api


In [2]:
host = "localhost"
port = "8100"

# T0PP

## Example outputs

In [52]:
get_answer(f"Simple definition of a hyperplane:")

'<pad> A hyperplane is a plane in three-dimensional space that is perpendicular to all other planes and tangent to all other lines.</s>'

In [53]:
get_answer(f"What mathematical concepts are mentioned here? A hyperplane is a plane in three-dimensional space that is perpendicular to all other planes and tangent to all other lines.")

'<pad> plane, line, space</s>'

In [57]:
get_answer(f"In mathematics, what is matrix?")

'<pad> a two-dimensional array of elements</s>'

In [74]:
get_answer(f"Give me a detailled definition of Kernel (algebra)")

'A kernel is a function that maps a real number to another real number.'

In [62]:
get_answer(f"In linear algebra, what is matrix transpose?")

'<pad> matrix transpose is a matrix operation that transposes a matrix into its identity matrix.</s>'

In [63]:
get_answer(f"Germany // Berlin\nFrance // Paris\nRussia // Moscow\nItaly - Rome\n USA //")

'<pad> Berlin</s>'

## Dep Extraction

In [4]:
def get_answer(prompt):
    url = f"http://{host}:{port}?prompt={prompt}"
    response = requests.request("GET", url)
    result = json.loads(response.text)
    return result['response'].replace("<pad> ","").replace("</s>", "")

In [5]:
def get_deps(i, concept):
    if i % 5 == 0: print(i)
    definition = get_answer(f"What is the mathematical definition of {concept}?")
    response = get_answer(f"What mathematical concepts are mentioned here: {definition}")
    deps = list(map(lambda x: x.strip(), response.split(",")))
    return deps

_, wiki_concepts = data_layer.read_index_and_wiki_concepts()
wiki_concepts = list(wiki_concepts)

#concepts = np.random.choice(wiki_concepts, 100)
deps = [get_deps(i, concept) for i, concept in enumerate(wiki_concepts)]
df_concepts = pd.DataFrame({"concept": wiki_concepts, "deps": deps})
df_concepts.to_csv("../dat/llm/t0pp_output.csv")

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
1095
1100
1105
1110
1115
1120
1125
1130
1135
1140
1145
1150
1155
1160
1165
1170
1175
1180
1185
1190
1195
1200
1205
1210
1215
12

In [55]:
#df_concepts = pd.read_csv("../dat/llm/t0pp_output.csv")
#import ast
#df_concepts.deps = df_concepts.deps.apply(ast.literal_eval)

In [6]:
link_set = set(wiki_concepts)
df_concepts['dep_articles'] = [print(i) or [wiki_api.disambiguate(dep, link_set, "Mathematics") for dep in concept_deps] for i, concept_deps in enumerate(df_concepts.deps)]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
df_concepts.to_csv("../dat/llm/df_concepts.csv")

In [8]:
graph_api.save_graph('../dat/llm/graph.json', df_concepts.concept, df_concepts.dep_articles)

In [9]:
print(list(zip(df_concepts.concept, df_concepts.deps, df_concepts.dep_articles)))

[('Additive identity', ['property', 'set', 'element'], ['Property (mathematics)', 'Set (mathematics)', 'Element (mathematics)']), ('Configuration space (mathematics)', ['set', 'configuration', 'system'], ['Set (mathematics)', 'Configuration (geometry)', 'Mathematics']), ('Coefficient of variation', ['a measure of the variability of a population'], ['Heart rate variability']), ('Continuum hypothesis', ['theory', 'function', 'continuous'], ['Set theory', 'Function (mathematics)', 'Continuous function']), ('Series–parallel graph', ['directed line segment', 'graph', 'vertex'], ['Line segment', 'Graph (discrete mathematics)', 'Vertex (geometry)']), ('Parallel (operator)', ['operator', 'parallel', 'perform'], ['Operator (mathematics)', 'Parallel (operator)', 'HP-01']), ('Subspace theorem', ['theorem', 'proof', 'concept'], ['Theorem', 'Proof theory', 'Concept']), ('Exotic sphere', ['surface', 'sphere', 'spherical'], ['Surface (mathematics)', 'Sphere', 'Spherical cap']), ('Coordinate system', 

# Galactica

In [4]:
def get_answer(prompt):
    url = f"http://{host}:{port}?prompt={prompt}"
    response = requests.request("GET", url)
    result = json.loads(response.text)
    return result['response'] #.replace("<pad> ","").replace("</s>", "")

In [7]:
get_answer(f"This is a list of mathematical concepts that are required to define matrix:\n")

['This is a list of mathematical concepts that are required to define matrix:\n\n* Set\n* Function\n* Cartesian product\n* Cartesian product of sets\n* Cartesian product of functions\n* Cartesian']

In [8]:
get_answer(f"Brief definition of matrix:\n")

['Brief definition of matrix:\n\nA matrix is a rectangular array of numbers, symbols, or expressions arranged in rows and columns.\n\n# 2.2.1.1 Matrix Addition']

In [10]:
get_answer(f"A matrix is a rectangular array of numbers, symbols, or expressions arranged in rows and columns.\n\nQ:What are the mathematical objects used in the above definition?\n\nA:")

['A matrix is a rectangular array of numbers, symbols, or expressions arranged in rows and columns.\n\nQ:What are the mathematical objects used in the above definition?\n\nA:Numbers']

In [19]:
get_answer(f"Brief definition of hyperplane:\n")

['Brief definition of hyperplane:\n\nA hyperplane is a set of points in a vector space that are equidistant from a given point.\n\n\nAnswer:\n\nA hyperplane is a subspace of']

In [22]:
get_answer(f"A hyperplane is a set of points in a vector space that are equidistant from a given point.\n\nQ: What three objects are mentioned in the abstract above?\n\nA:'")

["A hyperplane is a set of points in a vector space that are equidistant from a given point.\n\nQ: What three objects are mentioned in the abstract above?\n\nA:'Vector space"]

In [25]:
get_answer(f"This is a list of concepts in the definition of Convex set:\n\n")

['This is a list of concepts in the definition of Convex set:\n\n* \\(\\forall x,y\\in S:\\forall \\lambda \\in [0,']

In [66]:
indices, wiki_concepts = data_layer.read_index_and_wiki_concepts()

In [99]:
for concept in np.random.choice(wiki_concepts, 20):
    print("CONCEPT: " + concept)
    context = wiki_api.get_first_paragraph(concept)
    response = get_answer(f"{context}\n\nQ: What mathematical concepts are mentioned in the abstract above?\n\nA:")
    print(response)

CONCEPT: Boundary value problem
['In mathematics, in the field of differential equations, a boundary value problem is a differential equation together with a set of additional constraints, called the boundary conditions.[1] A solution to a boundary value problem is a solution to the differential equation which also satisfies the boundary conditions.\n\nQ: What mathematical concepts are mentioned in the abstract above?\n\nA: differential equations, boundary value problem']
CONCEPT: Minimal polynomial (linear algebra)
['In linear algebra, the minimal polynomial μA of an n × n matrix A over a field F is the monic polynomial P over F of least degree such that P(A) = 0.  Any other polynomial Q with Q(A) = 0 is a (polynomial) multiple of μA.\n\nQ: What mathematical concepts are mentioned in the abstract above?\n\nA: Polynomials, matrices, fields']
CONCEPT: Linear function (calculus)
['In calculus and related areas of mathematics, a linear function from the real numbers to the real numbers is